In [ ]:
import numpy as np
import pandas as pd

In [ ]:
books = pd.read_csv("clean_book.csv")

In [ ]:
books.columns

In [ ]:
books.categories.value_counts().shape

In [ ]:
books['categories'].value_counts().reset_index().query("count>=50")

In [ ]:
category_mapping ={
    'Fiction':"Fiction",
    "Juvenile Fiction": "Children's Fiction",
    "Biography & Autobiography": "NonFiction",
    "History":"NonFiction",
    "Literary Criticism":"NonFiction",
    "Philosophy":"NonFiction",
    "Religion":"NonFiction",
    "Comics & Graphic Novels":"Fiction",
    "Drama":"Fiction",
    "Science":"NonFiction",
    "Juvenile Nonfiction":"Children's NonFiction",
    "Poetry":"Fiction"
}

In [ ]:
books['modified_category'] = books['categories'].map(category_mapping)

In [ ]:
books[~books['modified_category'].isna()].shape

In [ ]:
from transformers import pipeline
pipe = pipeline("zero-shot-classification",model='facebook/bart-large-mnli')

In [ ]:
example1 = books.loc[books["modified_category"] == "Fiction","description"].reset_index(drop=True)[0]

In [ ]:
ex2=books.loc[books["modified_category"] == "NonFiction","description"].reset_index(drop=True)[0]

In [ ]:
category = ['Fiction' , 'NonFiction']

In [ ]:
pred1 = pipe(ex2 ,category)

In [ ]:
pred1

In [ ]:
pred1
pred1[np.argmax(pred1)]

In [ ]:
def generate_prediction(example , categories=['Fiction' , 'NonFiction']):
    pred = pipe(example,categories)
    index = np.argmax(pred['scores'])
    label = pred['labels'][index]

    return label

In [ ]:
generate_prediction(example1,category)

In [ ]:
from tqdm import tqdm

In [ ]:
isbn=[]
predicted_cats=[]

missing_categ = books.loc[books['modified_category'].isna(),['isbn13','description']].reset_index(drop=True)

In [ ]:
missing_categ

In [ ]:
for i in tqdm(range(0,len(missing_categ))):
    seq= missing_categ['description']
    predicted_cats += [generate_prediction(seq)]
    isbn+= [missing_categ['isbn13'][i]]

In [ ]:
df=pd.DataFrame({"isbn":isbn,"category":predicted_cats})

In [ ]:
df['category'].value_counts()